## View CTI results

In this notebook, we query the CTI results and plot them\
Craig Lage - 16-Sep-22

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.daf.butler import Butler

In [ ]:
def detector(raft, sensor):
    # Subroutine to find vendor and detector number given raft and sensor                                                                                                                                                           
    startingCol = [1,0,0,0,1] # First raft column in each row                                                                                                                                                                       
    rows = [0,3,8,13,18] # Starting raft sequence number of each row                                                                                                                                                                
    if raft in ['R11','R12','R13','R14','R21','R22','R23','R24','R30',\
                'R31','R32','R33','R34']:
        vendor = 'E2V'
    else:
        vendor = 'ITL'
    raftRow = int(list(raft)[1])
    raftCol = int(list(raft)[2])
    sensorRow = int(list(sensor)[1])
    sensorCol = int(list(sensor)[2])
    detectorNum = (rows[raftRow] + (raftCol - startingCol[raftRow])) * 9
    detectorNum += 3 * sensorRow + sensorCol
    return vendor, detectorNum

rafts = [       'R01', 'R02', 'R03', \
         'R10', 'R11', 'R12', 'R13', 'R14', \
         'R20', 'R21', 'R22', 'R23', 'R24', \
         'R30', 'R31', 'R32', 'R33', 'R34', \
                'R41', 'R42', 'R43']
sensors = ['S00', 'S01', 'S02', 'S10', 'S11', 'S12', 'S20', 'S21', 'S22']



In [ ]:
# Gen3 butler
REPO_DIR = '/repo/main'
butler = Butler(REPO_DIR, collections=["LSSTCam/raw/all","LSSTCam/calib", "u/cslage/sdf/BOT/cti_20220916"])
camera = butler.get('camera', instrument='LSSTCam')
exposure = 3021120600575

In [ ]:
ctiCalib = butler.get('cpCtiCalib', detector=55, exposure=exposure, instrument='LSSTCam')

In [ ]:
plt.figure(figsize = (16,16))
plotCounter = 1
for RAFT in rafts:
    for SENSOR in sensors:
        VENDOR, DETECTOR= detector(RAFT, SENSOR)
        if DETECTOR not in [55, 74]:
            continue
        ctiCalib = butler.get('cpCtiCalib', detector=DETECTOR, exposure=exposure, instrument='LSSTCam')
        plt.subplot(2,2,plotCounter)
        plt.title(f"CTI results, Detector {DETECTOR}, {RAFT}_{SENSOR}, {VENDOR}", fontsize = 18)
        for amp in camera[0].getAmplifiers():
            ampName = amp.getName()
            driftScale = ctiCalib.driftScale[ampName]
            decayTime = ctiCalib.decayTime[ampName]
            globalCti = ctiCalib.globalCti[ampName]
            trap = ctiCalib.serialTraps[ampName].coeffs
            trapLength = int(len(trap)/2)
            centers = trap[0:trapLength]
            values = trap[trapLength:2*trapLength]
            label = f"{ampName}, DriftScale={driftScale:.3e}, DecayTime={decayTime:.2f}, GlobalCTI={globalCti:.3e}"
            plt.scatter(centers, values, label=label)
            plt.xlabel("Flux(ADU?)")
            plt.ylabel("ADU??")
        plt.legend(bbox_to_anchor=(1.05, 0.8))
        plotCounter += 2
plt.savefig("/sdf/group/rubin/repo/main/u/cslage/sdf/BOT/cti_20220919/plots/CTI_19Sep22.pdf")


In [ ]:
# Gen3 butler
REPO_DIR = '/repo/main'
butler = Butler(REPO_DIR, collections=["LSSTCam/raw/all","LSSTCam/calib", "u/cslage/sdf/BOT/cti_test_11_20220919"])
camera = butler.get('camera', instrument='LSSTCam')
exposure = 3021120600575

In [ ]:
plt.figure(figsize = (16,8))
plotCounter = 1
for RAFT in rafts:
    for SENSOR in sensors:
        VENDOR, DETECTOR= detector(RAFT, SENSOR)
        if DETECTOR not in [11]:
            continue
        ctiCalib = butler.get('cpCtiCalib', detector=DETECTOR, exposure=exposure, instrument='LSSTCam')
        plt.subplot(1,2,plotCounter)
        plt.title(f"CTI results, Detector {DETECTOR}, {RAFT}_{SENSOR}, {VENDOR}", fontsize = 18)
        for amp in camera[0].getAmplifiers():
            ampName = amp.getName()
            driftScale = ctiCalib.driftScale[ampName]
            decayTime = ctiCalib.decayTime[ampName]
            globalCti = ctiCalib.globalCti[ampName]
            trap = ctiCalib.serialTraps[ampName].coeffs
            trapLength = int(len(trap)/2)
            centers = trap[0:trapLength]
            values = trap[trapLength:2*trapLength]
            label = f"{ampName}, DriftScale={driftScale:.3e}, DecayTime={decayTime:.2f}, GlobalCTI={globalCti:.3e}"
            plt.scatter(centers, values, label=label)
            plt.xlabel("Flux(ADU?)")
            plt.ylabel("ADU??")
        plt.legend(bbox_to_anchor=(1.05, 0.8))
        plotCounter += 2
plt.savefig("/sdf/group/rubin/repo/main/u/cslage/sdf/BOT/cti_20220919/plots/CTI_R02S02_19Sep22.pdf")


In [ ]:
amps=['C00','C01','C02','C03','C04','C05','C06','C07', \
      'C10','C11','C12','C13','C14','C15','C16','C17']
plt.figure(figsize = (16,16))
plt.subplots_adjust(hspace=0.3, wspace=0.3)
plotCounter = 1
for RAFT in rafts:
    for SENSOR in sensors:
        VENDOR, DETECTOR= detector(RAFT, SENSOR)
        if DETECTOR not in [11]:
            continue
        ctiCalib = butler.get('cpCtiCalib', detector=DETECTOR, exposure=exposure, instrument='LSSTCam')
        plt.subplot(4,4,plotCounter)
        #plt.title(f"CTI results, Detector {DETECTOR}, {RAFT}_{SENSOR}, {VENDOR}", fontsize = 18)
        for ampName in amps:
            plt.subplot(4,4,plotCounter)
            plt.title(f"{RAFT}_{SENSOR}_{ampName}")
            trap = ctiCalib.serialTraps[ampName].coeffs
            trapLength = int(len(trap)/2)
            centers = trap[0:trapLength]
            values = trap[trapLength:2*trapLength]
            plt.scatter(centers, values)
            plt.xlabel("Flux(ADU?)")
            plt.ylabel("Captured Charge(ADU?)")
            plt.xlim(0,150000)
            plt.xticks([0,50000,100000,150000])
            plt.ylim(0,1500)
            plotCounter += 1
plt.savefig("/sdf/group/rubin/repo/main/u/cslage/sdf/BOT/cti_20220919/plots/CTI_R02S02_ByAmp_19Sep22.png")


In [ ]:
amps=['C00','C01','C02','C03','C04','C05','C06','C07', \
      'C10','C11','C12','C13','C14','C15','C16','C17']
plt.figure(figsize = (16,16))
plt.subplots_adjust(hspace=0.3, wspace=0.3)
for RAFT in rafts:
    for SENSOR in sensors:
        VENDOR, DETECTOR= detector(RAFT, SENSOR)
        if DETECTOR not in [11]:
            continue
        ctiCalib = butler.get('cpCtiCalib', detector=DETECTOR, exposure=exposure, instrument='LSSTCam')
        Als = []
        Taus = []
        CTIs = []
        for ampName in amps:
            driftScale = ctiCalib.driftScale[ampName]
            decayTime = ctiCalib.decayTime[ampName]
            globalCti = ctiCalib.globalCti[ampName]
            Als.append(driftScale * 1.0E4)
            Taus.append(decayTime)
            CTIs.append(np.log10(globalCti))
            
        
        plt.subplot(1,3,1)
        plt.hist(Als)
        plt.xlim(0,10.0)
        plt.xlabel("Al*1E4")
        plt.subplot(1,3,2)
        plt.hist(Taus)
        plt.xlim(0,8.0)
        plt.xlabel("Tau(usec)")
        plt.subplot(1,3,3)
        plt.hist(CTIs)
        plt.xlim(-7.0,-4.0)
        plt.xticks([-7.0,-6.0,-5.0,-4.0])
        plt.xlabel("Log10 CTI")

plt.savefig("/sdf/group/rubin/repo/main/u/cslage/sdf/BOT/cti_20220919/plots/CTI_R02S02_Hist_19Sep22.png")


In [ ]:
# Find the CTI names:
names = []
for type in list(butler.registry.queryDatasetTypes()):
    names.append(type.name)
names.sort()
for name in names:
    print(name)
    
#cpCtiCalib
#cpCtiProc
#cpCtiStats

In [ ]:
ctiStats = butler.get('cpCtiStats', detector=11, exposure=3021120700221, instrument='LSSTCam')

In [ ]:
ctiStats['CTI']['C00']

In [ ]:
butler = Butler(REPO_DIR, collections=["LSSTCam/raw/all","LSSTCam/calib", "u/cslage/sdf/BOT/cti_20220916"])
ctiStats = butler.get('cpCtiStats', detector=55, exposure=3021120700221, instrument='LSSTCam')
print(ctiStats['CTI']['C00'])

In [ ]:
print(ctiStats['CTI']['C17'])

In [ ]:
filename = '/sdf/group/rubin/repo/main/u/cslage/sdf/BOT/cti_fix3_20220916/20220918T201358Z/cpCtiCalib/cpCtiCalib_LSSTCam_R20_S02_u_cslage_sdf_BOT_cti_fix3_20220916_20220918T201358Z.fits'
hdulist = pf.open(filename)

In [ ]:
plt.figure(figsize = (8,8))
for i in range(16):
    trap = hdulist[2].data[i][5]
    centers = trap[0:699]
    values = trap[699:1398]
    label = f"{amp}, DriftScale={driftScale:.3g}, DecayTime={decayTime:.2f}, GlobalCTI={globalCti:.2g}"
    plt.scatter(centers, values, label=label)
plt.legend()
    

In [ ]:
len(values)

In [ ]:
plt.scatter(centers, values)

In [ ]:
trap = hdulist[2].data[8][5]
for i in range(699):
    print(trap[i], trap[699+i])

In [ ]:
for i in range(1366, 1398):
    print(trap[i])

In [ ]:
filename = '/sdf/group/rubin/repo/main/u/cslage/sdf/BOT/cti_20220916/20220916T162216Z/cpCtiCalib/cpCtiCalib_LSSTCam_R20_S02_u_cslage_sdf_BOT_cti_20220916_20220916T162216Z.fits'
hdulist = pf.open(filename)

In [ ]:
hdulist[1].data

##### 

In [ ]:
ccds = ['R20_S02']
for ccd in ccds:
    print(f"CCD = {ccd}")
    filename = f'/sdf/group/rubin/repo/main/u/cslage/sdf/BOT/cti_fix3_20220916/20220918T201358Z/cpCtiCalib/cpCtiCalib_LSSTCam_{ccd}_u_cslage_sdf_BOT_cti_fix3_20220916_20220918T201358Z.fits'
    hdulist = pf.open(filename)
    print("AMP  len(table), numNans")
    for i in range(16):
        amp = hdulist[2].data[i][0]
        table = hdulist[2].data[i][5]
        numNans = 0
        for i in table:
            if np.isnan(i):
                numNans += 1
        print(amp, '     ', len(table),'     ', numNans)

In [ ]:
# Gen3 butler
REPO_DIR = '/repo/main'
butler = Butler(REPO_DIR, collections=["LSSTCam/raw/all","LSSTCam/calib", "u/cslage/sdf/BOT/cti_fix2_20220916"])
 

In [ ]:
exposure = 3021120600575
test = butler.get('cpCtiCalib', detector=74, exposure=exposure, instrument='LSSTCam')


In [ ]:
ccds = ['R13_S01', 'R20_S02']
for ccd in ccds:
    print(f"CCD = {ccd}")
    filename = f'/sdf/group/rubin/repo/main/u/cslage/sdf/BOT/cti_fix_20220916/20220916T181400Z/cpCtiCalib/cpCtiCalib_LSSTCam_{ccd}_u_cslage_sdf_BOT_cti_fix_20220916_20220916T181400Z.fits'
    hdulist = pf.open(filename)
    print("AMP  len(table), numNans")
    for i in range(16):
        amp = hdulist[2].data[i][0]
        table = hdulist[2].data[i][5]
        numNans = 0
        for i in table:
            if np.isnan(i):
                numNans += 1
        print(amp, '     ', len(table),'     ', numNans)